In [1]:
#Import required library

import pandas as pd
import numpy as  np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#Import prisoners data
URL_DATA = 'https://storage.data.gov.my/publicsafety/prisoners_state.parquet'

df = pd.read_parquet(URL_DATA)
if 'date' in df.columns: df['date'] = pd.to_datetime(df['date'])

prisoners = df

prisoners

,date,state,sex,prisoners
0,2017-01-01,Malaysia,both,120048
1,2018-01-01,Malaysia,both,115488
2,2019-01-01,Malaysia,both,136145
3,2020-01-01,Malaysia,both,100053
4,2021-01-01,Malaysia,both,82269
...,...,...,...,...
229,2018-01-01,Terengganu,male,4967
230,2019-01-01,Terengganu,male,5127
231,2020-01-01,Terengganu,male,4536
232,2021-01-01,Terengganu,male,4044


In [3]:
#Year column 
prisoners['year'] = prisoners['date'].dt.year

In [4]:
#both sex
prisoners_bsex = prisoners[prisoners['sex']=='both']
prisoners_bsex = prisoners_bsex.rename(columns={'prisoners':'sum_prisoners'})
prisoners_bsex

,date,state,sex,sum_prisoners,year
0,2017-01-01,Malaysia,both,120048,2017
1,2018-01-01,Malaysia,both,115488,2018
2,2019-01-01,Malaysia,both,136145,2019
3,2020-01-01,Malaysia,both,100053,2020
4,2021-01-01,Malaysia,both,82269,2021
...,...,...,...,...,...
217,2018-01-01,Terengganu,both,4967,2018
218,2019-01-01,Terengganu,both,5127,2019
219,2020-01-01,Terengganu,both,4551,2020
220,2021-01-01,Terengganu,both,4084,2021


In [5]:
#Get ratio for M:F for all states & add it to original prisoner df
prisoners_bsex_ratio = prisoners.merge(prisoners_bsex[['year','state','sum_prisoners']],how='left',on=['year','state'])
prisoners.insert(3,'sex_ratio',round((prisoners_bsex_ratio['prisoners']/prisoners_bsex_ratio['sum_prisoners']),2))
prisoners

,date,state,sex,sex_ratio,prisoners,year
0,2017-01-01,Malaysia,both,1.00,120048,2017
1,2018-01-01,Malaysia,both,1.00,115488,2018
2,2019-01-01,Malaysia,both,1.00,136145,2019
3,2020-01-01,Malaysia,both,1.00,100053,2020
4,2021-01-01,Malaysia,both,1.00,82269,2021
...,...,...,...,...,...,...
229,2018-01-01,Terengganu,male,1.00,4967,2018
230,2019-01-01,Terengganu,male,1.00,5127,2019
231,2020-01-01,Terengganu,male,1.00,4536,2020
232,2021-01-01,Terengganu,male,0.99,4044,2021


In [128]:
#Prepare data for map generation via power BI
#Set year to 2022
#Select only both gender 
#Remove Malaysia state

prisoners_both_msex = prisoners[(prisoners['year']==2022) & (prisoners['state']!='Malaysia') & (prisoners['sex']=='both')] 
prisoners_m_only = prisoners[(prisoners['sex']=='male') & (prisoners['year']==2022) & (prisoners['state']!='Malaysia')]
prisoners_both_msex = prisoners_both_msex.drop(columns='sex_ratio').merge(prisoners_m_only[['state','sex_ratio']],on='state',how='left').rename(columns={'sex_ratio':'m_sex_ratio'})
prisoners_adj_2022 = prisoners_both_msex.sort_values(by='prisoners', ascending=False).reset_index(drop=True)
prisoners_adj_2022


,date,state,sex,prisoners,year,m_sex_ratio
0,2022-01-01,Selangor,both,27765,2022,0.89
1,2022-01-01,Kedah,both,8980,2022,0.92
2,2022-01-01,Johor,both,8635,2022,0.93
3,2022-01-01,Sabah,both,8412,2022,0.92
4,2022-01-01,Perak,both,8276,2022,0.93
5,2022-01-01,Kelantan,both,7986,2022,0.91
6,2022-01-01,Pulau Pinang,both,6699,2022,0.93
7,2022-01-01,Sarawak,both,6089,2022,0.90
8,2022-01-01,Pahang,both,5977,2022,0.91
9,2022-01-01,Melaka,both,5281,2022,0.88


In [116]:
# Import population data
import pandas as pd

URL_DATA = 'https://storage.dosm.gov.my/population/population_state.parquet'

df = pd.read_parquet(URL_DATA)
if 'date' in df.columns: df['date'] = pd.to_datetime(df['date'])

population = df

population.head(5)

,state,date,sex,age,ethnicity,population
0,Johor,1970-01-01,both,overall,overall,1325.6
1,Johor,1970-01-01,both,0-4,overall,210.1
2,Johor,1970-01-01,both,5-9,overall,215.7
3,Johor,1970-01-01,both,10-14,overall,192.2
4,Johor,1970-01-01,both,15-19,overall,152.8


In [118]:
#put year column in population df
population['year'] = population['date'].dt.year

# Since, there's no data for Perlis column. Drop Perlis

population_adj = population.copy()
population_adj = population_adj[population_adj['state']!='Perlis']

# There's no WP columns in prisons data. KL & Putraya will be placed under Selangor whist Labuan placed under Sarawak
population_adj['state'] = population_adj['state'].replace({'W.P. Kuala Lumpur':'Selangor','W.P. Putrajaya':'Selangor','W.P. Labuan':'Sarawak'})

#Check whether the data is adjusted
print(population_adj['state'].value_counts())
population_adj

state
Selangor           39537
Sarawak            30582
Johor              17016
Kedah              17016
Kelantan           17016
Melaka             17016
Negeri Sembilan    17016
Pahang             17016
Perak              17016
Pulau Pinang       17016
Sabah              17016
Terengganu         17016
Name: count, dtype: int64


,state,date,sex,age,ethnicity,population,year
0,Johor,1970-01-01,both,overall,overall,1325.6,1970
1,Johor,1970-01-01,both,0-4,overall,210.1,1970
2,Johor,1970-01-01,both,5-9,overall,215.7,1970
3,Johor,1970-01-01,both,10-14,overall,192.2,1970
4,Johor,1970-01-01,both,15-19,overall,152.8,1970
...,...,...,...,...,...,...,...
257290,Selangor,2024-01-01,male,85+,bumi_other,0.0,2024
257291,Selangor,2024-01-01,male,85+,chinese,0.0,2024
257292,Selangor,2024-01-01,male,85+,indian,0.0,2024
257293,Selangor,2024-01-01,male,85+,other_citizen,0.0,2024


In [151]:
#filter for 2022 and select all for sex, age, ethinicity
population_adj_22 = population_adj[(population_adj['year']==2022) & (population_adj['sex']=='both') & (population_adj['ethnicity']=='overall') & (population_adj['age'] == 'overall')]
population_adj_22 = population_adj_22.groupby('state').agg(population = ('population','sum')).sort_values(by='population',ascending=False).reset_index()
population_adj_22

,state,population
0,Selangor,9128.5
1,Johor,4028.3
2,Sabah,3414.9
3,Sarawak,2570.4
4,Perak,2514.4
5,Kedah,2163.1
6,Kelantan,1830.6
7,Pulau Pinang,1740.9
8,Pahang,1614.3
9,Negeri Sembilan,1207.9


In [153]:
merged = prisoners_adj_2022.merge(population_adj_22[['state','population']],on='state',how='left')
merged['inc_rate_per_100k'] = merged['prisoners']/((merged['population']*1000)/100000)
fin_merged = merged.drop(columns=['date','sex','year'])
fin_merged

,state,prisoners,m_sex_ratio,population,inc_rate_per_100k
0,Selangor,27765,0.89,9128.5,304.157310
1,Kedah,8980,0.92,2163.1,415.144931
2,Johor,8635,0.93,4028.3,214.358414
3,Sabah,8412,0.92,3414.9,246.332250
4,Perak,8276,0.93,2514.4,329.144130
5,Kelantan,7986,0.91,1830.6,436.250410
6,Pulau Pinang,6699,0.93,1740.9,384.800965
7,Sarawak,6089,0.90,2570.4,236.889200
8,Pahang,5977,0.91,1614.3,370.253361
9,Melaka,5281,0.88,1008.6,523.597065


In [157]:
fin_merged.to_csv('D:/OneDrive/WF - Personal/WF/JobHunt/Data Analyst/LinkedIn posting/Prisoners Analysis/fin_merged.csv')

In [199]:
print((fin_merged['population'].sum()*1000)/100000)
print(fin_merged['prisoners'].sum())
(fin_merged['prisoners'].sum())/((fin_merged['population'].sum()*1000)/100000)

324.085
101809


314.1428946109817

101809